<a href="https://colab.research.google.com/github/pashkev14/ML_Labs/blob/main/Lab_3_2373_Savin_PA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv('lab_dataset_processed.csv')

In [2]:
df = df.drop(columns='Unnamed: 0', axis=1)
df.head()

,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,0,98,34.52,69.18,104,1715849100,2024-05-16 13:15,26.6,79.8,18,...,8.4,26.6,1,1,04:50 AM,06:50 PM,12:12 PM,01:11 AM,7,55
1,1,225,41.33,19.82,166,1715849100,2024-05-16 10:45,19.0,66.2,19,...,1.1,2.0,1,1,05:21 AM,07:54 PM,12:58 PM,02:14 AM,7,55
2,2,11,36.76,3.05,2,1715849100,2024-05-16 09:45,23.0,73.4,26,...,10.4,18.4,1,1,05:40 AM,07:50 PM,01:15 PM,02:14 AM,7,55
3,3,14,42.50,1.52,133,1715849100,2024-05-16 10:45,6.3,43.3,6,...,0.7,0.9,1,1,06:31 AM,09:11 PM,02:12 PM,03:31 AM,7,55
4,4,122,-8.84,13.23,27,1715849100,2024-05-16 09:45,26.0,78.8,19,...,183.4,262.3,5,10,06:12 AM,05:55 PM,01:17 PM,12:38 AM,7,55


Заменяем колонки sunrise и sunset на разницу между ними в секундах, так как модели не могут работать с таким типом признаков.

In [3]:
import datetime as dt

def diff_between_sunrise_and_sunset(sunrise, sunset):

  def convert_into_seconds(value):
    values = value.split()
    time = dt.datetime.strptime(values[0], '%H:%M')
    if values[1] == 'PM':
      time += dt.timedelta(hours=12)
    return (time.hour * 3600) + (time.minute * 60)

  sunrise = convert_into_seconds(sunrise)
  sunset = convert_into_seconds(sunset)
  return sunset - sunrise

df['day_duration_s'] = df.apply(lambda x: diff_between_sunrise_and_sunset(x['sunrise'], x['sunset']), axis=1)

df = df.drop(columns=['sunrise', 'sunset', 'moonrise', 'moonset'], axis=1)
df.head()

,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_Ozone,air_quality_Nitrogen_dioxide,air_quality_Sulphur_dioxide,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,moon_phase,moon_illumination,day_duration_s
0,0,98,34.52,69.18,104,1715849100,2024-05-16 13:15,26.6,79.8,18,...,103.0,1.1,0.2,8.4,26.6,1,1,7,55,50400
1,1,225,41.33,19.82,166,1715849100,2024-05-16 10:45,19.0,66.2,19,...,97.3,0.9,0.1,1.1,2.0,1,1,7,55,52380
2,2,11,36.76,3.05,2,1715849100,2024-05-16 09:45,23.0,73.4,26,...,12.2,65.1,13.4,10.4,18.4,1,1,7,55,51000
3,3,14,42.50,1.52,133,1715849100,2024-05-16 10:45,6.3,43.3,6,...,64.4,1.6,0.2,0.7,0.9,1,1,7,55,52800
4,4,122,-8.84,13.23,27,1715849100,2024-05-16 09:45,26.0,78.8,19,...,19.0,72.7,31.5,183.4,262.3,5,10,7,55,42180


In [4]:
print(df.dtypes)

country                           int64
location_name                     int64
latitude                        float64
longitude                       float64
timezone                          int64
last_updated_epoch                int64
last_updated                     object
temperature_celsius             float64
temperature_fahrenheit          float64
condition_text                    int64
wind_mph                        float64
wind_kph                        float64
wind_degree                       int64
wind_direction                    int64
pressure_mb                     float64
pressure_in                     float64
precip_mm                       float64
precip_in                       float64
humidity                          int64
cloud                             int64
feels_like_celsius              float64
feels_like_fahrenheit           float64
visibility_km                   float64
visibility_miles                float64
uv_index                        float64


In [5]:
df = df.drop(columns='last_updated', axis=1)

В целях сокращения сложных вычислений удаляем признаки, которые, по сути, дублируют друг друга (например, температура в градусах Цельсия и Фаренгейта).

In [6]:
empiric_metrics = ['temperature_fahrenheit',
                   'wind_mph', 'pressure_in', 'precip_in', 'feels_like_fahrenheit', 'visibility_miles', 'gust_mph']
df = df.drop(columns=empiric_metrics, axis=1)

Признак, по которому будем классифицировать - moon_phase.

In [7]:
# kNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

class_feature = df['moon_phase']
df = df.drop(columns='moon_phase', axis=1)

x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(
    df, class_feature, test_size=0.95)

# kNN
model_knn = KNeighborsClassifier(n_neighbors=8)
model_knn.fit(x_train_data, y_train_data)
predictions_knn = model_knn.predict(x_test_data)

# DecisionTree
model_tree = DecisionTreeClassifier()
model_tree.fit(x_train_data, y_train_data)
predictions_tree = model_tree.predict(x_test_data)

In [8]:
class_feature.unique()

array([7, 1, 5, 2, 4, 3, 6, 0])

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def get_scores(y_test_data, predictions):
    accuracy = accuracy_score(y_test_data, predictions)
    precision = precision_score(y_test_data, predictions, average='weighted')
    recall = recall_score(y_test_data, predictions, average='weighted')
    f1 = f1_score(y_test_data, predictions, average='weighted')
    y_pred = model_knn.predict_proba(x_test_data)
    roc_auc = roc_auc_score(y_test_data, y_pred, multi_class='ovo')
    return {'Accuracy' : accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'ROC': roc_auc}


# KNN
scores_knn = get_scores(y_test_data, predictions_knn)
# DecisionTree
scores_tree = get_scores(y_test_data, predictions_tree)

print('KNN')
print(scores_knn)
print()
print('tree')
print(scores_tree)

KNN
{'Accuracy': 0.9999620781190747, 'Precision': 0.9999620843737144, 'Recall': 0.9999620781190747, 'F1 Score': 0.9999620693386517, 'ROC': 0.999931235826665}

tree
{'Accuracy': 0.9786499810390595, 'Precision': 0.9795051307183614, 'Recall': 0.9786499810390595, 'F1 Score': 0.9789758942510317, 'ROC': 0.999931235826665}


In [10]:
x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(
    df, class_feature, test_size=0.5)

# kNN
model_knn = KNeighborsClassifier(n_neighbors=8)
model_knn.fit(x_train_data, y_train_data)
predictions_knn = model_knn.predict(x_test_data)

# DecisionTree
model_tree = DecisionTreeClassifier()
model_tree.fit(x_train_data, y_train_data)
predictions_tree = model_tree.predict(x_test_data)

# KNN
scores_knn = get_scores(y_test_data, predictions_knn)
# DecisionTree
scores_tree = get_scores(y_test_data, predictions_tree)

print('KNN')
print(scores_knn)
print()
print('tree')
print(scores_tree)

KNN
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1 Score': 1.0, 'ROC': 1.0}

tree
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1 Score': 1.0, 'ROC': 1.0}


Полученные результаты очень любопытные. По всем показателям обе модели великолепно отработали, точность на грани фантастики.  
В качестве эксперимента рассмотрели разное соотношение тренировочной и тестовой выборок. При соотношении 5:95 лучше отработал алгоритм kNN. При соотношении 1:1 показатели дошли до идеала в обоих алгоритмах.